In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
!pip install keras --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [11]:
data_dir = '/content/drive/MyDrive/asl_alphabet_train/asl_alphabet_train'

In [12]:
Name=os.listdir(data_dir)
print(Name)
print(len(Name))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'space', 'nothing']
29


In [13]:
data_image=[]
data_label=[]
count=0
for name in tqdm(Name):
    path=os.path.join(data_dir,name)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(50,50))
        image=img_to_array(image)
        image=image/255.0
        data_image+=[image]
        data_label+=[count]
    count+=1

100%|██████████| 29/29 [26:20<00:00, 54.48s/it]


In [14]:
m=len(data_image)
M=list(range(m))
random.seed(2021)
random.shuffle(M)
print(m)

89097


In [15]:
data_image=np.array(data_image)
data_label=np.array(data_label)

In [17]:
trainX=data_image[M[0:(m//4)*3]]
trainY0=data_label[M[0:(m//4)*3]]
testX=data_image[M[(m//4)*3:]]
testY0=data_label[M[(m//4)*3:]]

N=list(range(len(Name)))    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name))

In [18]:
trainY0=pd.Series(trainY0)
testY0=pd.Series(testY0)

In [19]:
labels1=to_categorical(trainY0)
trainY=np.array(labels1)
trainx,testx,trainy,testy=train_test_split(trainX,trainY,test_size=0.2,random_state=44)

In [20]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(53457, 50, 50, 3)
(13365, 50, 50, 3)
(53457, 29)
(13365, 29)


In [21]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.6,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [22]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(50,50,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


74836368/74836368 [==============================] - 4s 0us/step


In [24]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(29, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [39]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
his=model.fit(datagen.flow(trainx,trainy,batch_size=16),validation_data=(testx,testy),epochs=100)

Epoch 1/100
3342/3342 [==============================] - ETA: 0s - batch: 1670.5000 - size: 15.9955 - loss: 2.1635 - acc: 0.3605

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


3342/3342 [==============================] - 172s 50ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1635 - acc: 0.3605 - val_loss: 2.8671 - val_acc: 0.2426
Epoch 2/100
3342/3342 [==============================] - 163s 49ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1463 - acc: 0.3620 - val_loss: 2.7541 - val_acc: 0.2519
Epoch 3/100
3342/3342 [==============================] - 164s 49ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1530 - acc: 0.3639 - val_loss: 2.6520 - val_acc: 0.2625
Epoch 4/100
3342/3342 [==============================] - 165s 49ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1471 - acc: 0.3661 - val_loss: 2.6844 - val_acc: 0.2530
Epoch 5/100
3342/3342 [==============================] - 164s 49ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1488 - acc: 0.3638 - val_loss: 2.7205 - val_acc: 0.2587
Epoch 6/100
3342/3342 [==============================] - 163s 49ms/step - batch: 1670.5000 - size: 15.9955 - loss: 2.1475 - acc: 0.3623 - val_loss:

In [40]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

           0       0.10      0.80      0.17       467
           1       0.15      0.30      0.20       519
           2       0.47      0.59      0.52       480
           3       0.30      0.21      0.24       460
           4       0.17      0.11      0.13       447
           5       0.88      0.33      0.48       448
           6       0.62      0.14      0.23       458
           7       0.44      0.45      0.45       454
           8       0.45      0.17      0.24       482
           9       0.29      0.43      0.35       526
          10       0.17      0.03      0.06       445
          11       0.16      0.41      0.23       441
          12       0.28      0.21      0.24       435
          13       0.11      0.09      0.10       460
          14       0.60      0.59      0.59       451
          15       0.32      0.61      0.42       448
          16       0.49      0.26      0.34       464
          17       0.27    